Let's say we want to prepare data and try some scalers and classifiers for prediction in a classification problem. We will tune paramaters of classifiers by grid search technique.

Data preparing:

In [1]:
from sklearn.datasets import make_classification


X, y = make_classification()

Setting steps for our pipelines and parameters for grid search:

In [2]:
from reskit.core import Pipeliner


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


classifiers = [('LR', LogisticRegression()),
               ('SVC', SVC())]

scalers = [('standard', StandardScaler()),
           ('minmax', MinMaxScaler())]

steps = [('scaler', scalers),
         ('classifier', classifiers)]

param_grid = {'LR': {'penalty': ['l1', 'l2']},
              'SVC': {'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}}

Creating a plan of our research:

In [3]:
pipe = Pipeliner(steps, param_grid=param_grid)
pipe.plan_table

,scaler,classifier
0,standard,LR
1,standard,SVC
2,minmax,LR
3,minmax,SVC


To tune parameters of models and evaluate this models, run:

In [4]:
pipe.get_results(X, y, scoring=['roc_auc'])

Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4


,scaler,classifier,grid_roc_auc_mean,grid_roc_auc_std,grid_roc_auc_best_params,eval_roc_auc_mean,eval_roc_auc_std,eval_roc_auc_scores
0,standard,LR,0.970294,0.0120079,{'penalty': 'l2'},0.97057,0.0120488,[ 0.97231834 0.9550173 0.984375 ]
1,standard,SVC,0.964191,0.00721387,{'kernel': 'sigmoid'},0.964357,0.00723858,[ 0.9550173 0.96539792 0.97265625]
2,minmax,LR,0.965294,0.0102643,{'penalty': 'l1'},0.965362,0.0101745,[ 0.95155709 0.97577855 0.96875 ]
3,minmax,SVC,0.965294,0.0102643,{'kernel': 'linear'},0.965362,0.0101745,[ 0.97577855 0.95155709 0.96875 ]
